# Partie 2.2 - Fusion et enrichissement des donnees

Ce notebook realise la fusion de toutes les sources de donnees nettoyees et l'enrichissement
avec des features derivees pour l'analyse energetique des batiments publics.

**Sources de donnees :**
- Consommations nettoyees (Parquet partitionne)
- Donnees meteorologiques nettoyees (CSV)
- Referentiel batiments (CSV)
- Tarifs energie (CSV)

**Sortie :** Un dataset enrichi pret pour l'analyse et la modelisation.

In [1]:
# --- Imports ---
import pandas as pd
import numpy as np
import os
import pyarrow.parquet as pq
from datetime import datetime
import warnings

warnings.filterwarnings('ignore')

# Configuration d'affichage
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

# Chemins relatifs vers les donnees
PATH_CONSO_CLEAN = '../output/consommations_clean/'
PATH_METEO_CLEAN = '../output/meteo_clean.csv'
PATH_BATIMENTS = '../data/batiments.csv'
PATH_TARIFS = '../data/tarifs_energie.csv'
PATH_OUTPUT = '../output/'

print("Imports et configuration OK")
print(f"Pandas version : {pd.__version__}")
print(f"NumPy version  : {np.__version__}")

Imports et configuration OK
Pandas version : 2.3.3
NumPy version  : 2.2.6


## 1. Chargement des donnees nettoyees

Chargement des 4 sources de donnees. Pour les consommations (~7M lignes en Parquet partitionne),
nous chargeons d'abord un echantillon pour validation, puis le dataset complet si la memoire le permet.

In [2]:
# ============================================================
# 1.1 Chargement du referentiel batiments
# ============================================================
df_batiments = pd.read_csv(PATH_BATIMENTS)
print(f"Batiments : {df_batiments.shape[0]} lignes, {df_batiments.shape[1]} colonnes")
print(f"Colonnes  : {list(df_batiments.columns)}")
display(df_batiments.head())

# ============================================================
# 1.2 Chargement des tarifs energie
# ============================================================
df_tarifs = pd.read_csv(PATH_TARIFS, parse_dates=['date_debut', 'date_fin'])
print(f"\nTarifs : {df_tarifs.shape[0]} lignes, {df_tarifs.shape[1]} colonnes")
print(f"Colonnes : {list(df_tarifs.columns)}")
display(df_tarifs.head())

# ============================================================
# 1.3 Chargement des donnees meteo nettoyees
# ============================================================
df_meteo = pd.read_csv(PATH_METEO_CLEAN, parse_dates=['timestamp'])
print(f"\nMeteo : {df_meteo.shape[0]} lignes, {df_meteo.shape[1]} colonnes")
print(f"Colonnes : {list(df_meteo.columns)}")
print(f"Communes : {df_meteo['commune'].unique()}")
print(f"Plage temporelle : {df_meteo['timestamp'].min()} -> {df_meteo['timestamp'].max()}")
display(df_meteo.head())

# ============================================================
# 1.4 Chargement des consommations nettoyees (Parquet partitionne)
# ============================================================
# Verification de la structure du repertoire Parquet
if os.path.exists(PATH_CONSO_CLEAN):
    print(f"\nRepertoire Parquet : {PATH_CONSO_CLEAN}")
    partitions = [d for d in os.listdir(PATH_CONSO_CLEAN) if os.path.isdir(os.path.join(PATH_CONSO_CLEAN, d))]
    print(f"Nombre de partitions : {len(partitions)}")
    if partitions:
        print(f"Exemples de partitions : {sorted(partitions)[:5]}")

# Lecture du schema Parquet pour inspection
try:
    pq_dataset = pq.ParquetDataset(PATH_CONSO_CLEAN)
    print(f"Schema Parquet : {pq_dataset.schema}")
except Exception as e:
    print(f"Note : Lecture schema Parquet - {e}")

# Chargement complet du Parquet
# Pour un dataset de ~7M lignes, on charge progressivement
try:
    # Tentative de lecture complete
    df_conso = pd.read_parquet(PATH_CONSO_CLEAN)
    print(f"\nConsommations chargees : {df_conso.shape[0]:,} lignes, {df_conso.shape[1]} colonnes")
except Exception as e:
    print(f"Lecture Parquet echouee ({e}), tentative CSV de secours...")
    # Fallback : lecture depuis le CSV brut avec echantillonnage
    df_conso = pd.read_csv(
        '../data/consommations_raw.csv',
        parse_dates=['timestamp']
    )
    print(f"Consommations chargees depuis CSV : {df_conso.shape[0]:,} lignes")

# Conversion du timestamp si necessaire
if 'timestamp' in df_conso.columns:
    df_conso['timestamp'] = pd.to_datetime(df_conso['timestamp'])

print(f"Colonnes consommations : {list(df_conso.columns)}")
print(f"Types d'energie : {df_conso['type_energie'].unique()}")
print(f"Batiments uniques : {df_conso['batiment_id'].nunique()}")
print(f"Plage temporelle : {df_conso['timestamp'].min()} -> {df_conso['timestamp'].max()}")
print(f"\nMemoire utilisee : {df_conso.memory_usage(deep=True).sum() / 1e6:.1f} Mo")
display(df_conso.head())

Batiments : 146 lignes, 8 colonnes
Colonnes  : ['batiment_id', 'nom', 'type', 'commune', 'surface_m2', 'annee_construction', 'classe_energetique', 'nb_occupants_moyen']


,batiment_id,nom,type,commune,surface_m2,annee_construction,classe_energetique,nb_occupants_moyen
0,BAT0001,Ecole Paris 1,ecole,Paris,1926,1978,E,225
1,BAT0002,Ecole Paris 2,ecole,Paris,1156,2004,C,402
2,BAT0003,Ecole Paris 3,ecole,Paris,1695,2014,D,219
3,BAT0004,Mediatheque Paris 4,mediatheque,Paris,907,2019,C,121
4,BAT0005,Piscine Paris 5,piscine,Paris,3913,1950,G,242



Tarifs : 10 lignes, 4 colonnes
Colonnes : ['date_debut', 'date_fin', 'type_energie', 'tarif_unitaire']


,date_debut,date_fin,type_energie,tarif_unitaire
0,2023-01-01,2023-06-30,electricite,0.1800
1,2023-07-01,2023-12-31,electricite,0.2000
2,2023-01-01,2023-06-30,gaz,0.0900
3,2023-07-01,2023-12-31,gaz,0.1000
4,2023-01-01,2023-12-31,eau,3.5000



Meteo : 252612 lignes, 12 colonnes
Colonnes : ['commune', 'timestamp', 'temperature_c', 'humidite_pct', 'rayonnement_solaire_wm2', 'vitesse_vent_kmh', 'precipitation_mm', 'jour', 'mois', 'heure', 'saison', 'jour_semaine']
Communes : ['Bordeaux' 'Le Havre' 'Lille' 'Lyon' 'Marseille' 'Montpellier' 'Nantes'
 'Nice' 'Paris' 'Reims' 'Rennes' 'Saint-Etienne' 'Strasbourg' 'Toulon'
 'Toulouse']
Plage temporelle : 2023-01-01 00:00:00 -> 2024-12-31 23:00:00


,commune,timestamp,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm,jour,mois,heure,saison,jour_semaine
0,Bordeaux,2023-01-01 00:00:00,8.5000,43.9000,0.8000,0.2000,0.0000,1,1,0,hiver,Dimanche
1,Bordeaux,2023-01-01 01:00:00,2.7000,39.7000,6.1000,21.5000,5.3000,1,1,1,hiver,Dimanche
2,Bordeaux,2023-01-01 02:00:00,6.2000,78.9000,49.5000,13.1000,0.0000,1,1,2,hiver,Dimanche
3,Bordeaux,2023-01-01 03:00:00,10.3000,64.2000,24.3000,0.6000,14.7000,1,1,3,hiver,Dimanche
4,Bordeaux,2023-01-01 04:00:00,0.9000,36.4000,20.7000,35.6000,0.0000,1,1,4,hiver,Dimanche



Repertoire Parquet : ../output/consommations_clean/
Nombre de partitions : 24
Exemples de partitions : ['annee_mois=2023-01', 'annee_mois=2023-02', 'annee_mois=2023-03', 'annee_mois=2023-04', 'annee_mois=2023-05']
Schema Parquet : batiment_id: string
unite: string
timestamp: timestamp[ns]
consommation: double
annee: int32
mois: int32
jour: int32
heure: int32
date: date32[day]
annee_mois: dictionary<values=string, indices=int32, ordered=0>
type_energie: dictionary<values=string, indices=int32, ordered=0>
-- schema metadata --
org.apache.spark.version: '4.1.1'
org.apache.spark.sql.parquet.row.metadata: '{"type":"struct","fields":[{"' + 585

Consommations chargees : 7,491,733 lignes, 11 colonnes
Colonnes consommations : ['batiment_id', 'unite', 'timestamp', 'consommation', 'annee', 'mois', 'jour', 'heure', 'date', 'annee_mois', 'type_energie']
Types d'energie : ['eau', 'electricite', 'gaz']
Categories (3, object): ['eau', 'electricite', 'gaz']
Batiments uniques : 146
Plage temporelle : 2

,batiment_id,unite,timestamp,consommation,annee,mois,jour,heure,date,annee_mois,type_energie
0,BAT0001,m3,2023-01-01 02:00:00,0.2600,2023,1,1,3,2023-01-01,2023-01,eau
1,BAT0001,m3,2023-01-01 17:00:00,2.2700,2023,1,1,18,2023-01-01,2023-01,eau
2,BAT0001,m3,2023-01-01 22:00:00,0.2000,2023,1,1,23,2023-01-01,2023-01,eau
3,BAT0001,m3,2023-01-02 21:00:00,14.6400,2023,1,2,22,2023-01-02,2023-01,eau
4,BAT0001,m3,2023-01-03 08:00:00,14.8100,2023,1,3,9,2023-01-03,2023-01,eau


## 2. Fusion consommations-batiments

Jointure gauche (left join) des consommations avec le referentiel batiments sur `batiment_id`.
Cela permet d'associer a chaque mesure de consommation les caracteristiques du batiment
(type, commune, surface, classe energetique, etc.).

In [3]:
# ============================================================
# Fusion consommations + batiments (left join sur batiment_id)
# ============================================================
print("--- Avant fusion ---")
print(f"Consommations : {df_conso.shape[0]:,} lignes")
print(f"Batiments     : {df_batiments.shape[0]} lignes")

# Verification des cles de jointure
conso_bat_ids = set(df_conso['batiment_id'].unique())
bat_ids = set(df_batiments['batiment_id'].unique())
orphelins = conso_bat_ids - bat_ids
print(f"\nBatiments dans consommations : {len(conso_bat_ids)}")
print(f"Batiments dans referentiel   : {len(bat_ids)}")
print(f"Batiments orphelins (dans conso mais pas dans ref.) : {len(orphelins)}")
if orphelins:
    print(f"  -> IDs orphelins : {sorted(orphelins)[:10]}")

# Realisation de la jointure
df_merged = df_conso.merge(
    df_batiments,
    on='batiment_id',
    how='left',
    validate='m:1',  # Chaque consommation correspond a un seul batiment
    indicator=True
)

# Verification de la qualite de la jointure
print(f"\n--- Apres fusion consommations-batiments ---")
print(f"Shape : {df_merged.shape}")
print(f"\nRepartition de la jointure :")
print(df_merged['_merge'].value_counts())

# Verification : pas de perte de lignes
assert df_merged.shape[0] == df_conso.shape[0], "ERREUR : Nombre de lignes modifie apres jointure !"
print(f"\nVerification OK : {df_merged.shape[0]:,} lignes conservees (aucune perte)")

# Suppression de la colonne indicateur
df_merged.drop(columns=['_merge'], inplace=True)

# Apercu du resultat
print(f"\nColonnes apres fusion : {list(df_merged.columns)}")
display(df_merged.head())

--- Avant fusion ---
Consommations : 7,491,733 lignes
Batiments     : 146 lignes

Batiments dans consommations : 146
Batiments dans referentiel   : 146
Batiments orphelins (dans conso mais pas dans ref.) : 0

--- Apres fusion consommations-batiments ---
Shape : (7491733, 19)

Repartition de la jointure :
_merge
both          7491733
left_only           0
right_only          0
Name: count, dtype: int64

Verification OK : 7,491,733 lignes conservees (aucune perte)

Colonnes apres fusion : ['batiment_id', 'unite', 'timestamp', 'consommation', 'annee', 'mois', 'jour', 'heure', 'date', 'annee_mois', 'type_energie', 'nom', 'type', 'commune', 'surface_m2', 'annee_construction', 'classe_energetique', 'nb_occupants_moyen']


,batiment_id,unite,timestamp,consommation,annee,mois,jour,heure,date,annee_mois,type_energie,nom,type,commune,surface_m2,annee_construction,classe_energetique,nb_occupants_moyen
0,BAT0001,m3,2023-01-01 02:00:00,0.2600,2023,1,1,3,2023-01-01,2023-01,eau,Ecole Paris 1,ecole,Paris,1926,1978,E,225
1,BAT0001,m3,2023-01-01 17:00:00,2.2700,2023,1,1,18,2023-01-01,2023-01,eau,Ecole Paris 1,ecole,Paris,1926,1978,E,225
2,BAT0001,m3,2023-01-01 22:00:00,0.2000,2023,1,1,23,2023-01-01,2023-01,eau,Ecole Paris 1,ecole,Paris,1926,1978,E,225
3,BAT0001,m3,2023-01-02 21:00:00,14.6400,2023,1,2,22,2023-01-02,2023-01,eau,Ecole Paris 1,ecole,Paris,1926,1978,E,225
4,BAT0001,m3,2023-01-03 08:00:00,14.8100,2023,1,3,9,2023-01-03,2023-01,eau,Ecole Paris 1,ecole,Paris,1926,1978,E,225


## 3. Fusion avec les donnees meteo

Jointure avec les donnees meteorologiques sur `commune` et `timestamp` (arrondi a l'heure).
Cela permet d'associer les conditions climatiques a chaque mesure de consommation.

In [4]:
# ============================================================
# Preparation des timestamps pour la jointure meteo
# Arrondi a l'heure pour aligner les deux sources
# ============================================================
df_merged['timestamp_heure'] = df_merged['timestamp'].dt.floor('h')
df_meteo['timestamp_heure'] = df_meteo['timestamp'].dt.floor('h')

print(f"Communes dans les consommations : {sorted(df_merged['commune'].unique())}")
print(f"Communes dans la meteo          : {sorted(df_meteo['commune'].unique())}")

# Communes communes aux deux datasets
communes_communes = set(df_merged['commune'].unique()) & set(df_meteo['commune'].unique())
print(f"Communes en commun : {sorted(communes_communes)}")

# Colonnes meteo a joindre (sans doublons avec le dataset principal)
colonnes_meteo = ['commune', 'timestamp_heure', 'temperature_c', 'humidite_pct',
                   'rayonnement_solaire_wm2', 'vitesse_vent_kmh', 'precipitation_mm',
                   'saison']

# Deduplication de la meteo sur (commune, timestamp_heure) pour eviter les doublons
df_meteo_dedup = df_meteo[colonnes_meteo].drop_duplicates(
    subset=['commune', 'timestamp_heure'],
    keep='first'
)
print(f"\nMeteo dedup : {df_meteo_dedup.shape[0]:,} lignes")

# Jointure gauche sur commune + timestamp_heure
nb_avant = df_merged.shape[0]
df_merged = df_merged.merge(
    df_meteo_dedup,
    on=['commune', 'timestamp_heure'],
    how='left'
)

# Verification de la qualite de la jointure
nb_apres = df_merged.shape[0]
nb_match = df_merged['temperature_c'].notna().sum()
taux_match = nb_match / nb_apres * 100

print(f"\n--- Apres fusion avec meteo ---")
print(f"Shape : {df_merged.shape}")
print(f"Lignes avant  : {nb_avant:,}")
print(f"Lignes apres  : {nb_apres:,}")
print(f"Lignes avec meteo : {nb_match:,} ({taux_match:.1f}%)")
print(f"Lignes sans meteo : {nb_apres - nb_match:,} ({100 - taux_match:.1f}%)")

# Nettoyage de la colonne temporaire
df_merged.drop(columns=['timestamp_heure'], inplace=True)

print(f"\nColonnes apres fusion meteo : {list(df_merged.columns)}")
display(df_merged.head())

Communes dans les consommations : ['Bordeaux', 'Le Havre', 'Lille', 'Lyon', 'Marseille', 'Montpellier', 'Nantes', 'Nice', 'Paris', 'Reims', 'Rennes', 'Saint-Etienne', 'Strasbourg', 'Toulon', 'Toulouse']
Communes dans la meteo          : ['Bordeaux', 'Le Havre', 'Lille', 'Lyon', 'Marseille', 'Montpellier', 'Nantes', 'Nice', 'Paris', 'Reims', 'Rennes', 'Saint-Etienne', 'Strasbourg', 'Toulon', 'Toulouse']
Communes en commun : ['Bordeaux', 'Le Havre', 'Lille', 'Lyon', 'Marseille', 'Montpellier', 'Nantes', 'Nice', 'Paris', 'Reims', 'Rennes', 'Saint-Etienne', 'Strasbourg', 'Toulon', 'Toulouse']

Meteo dedup : 236,232 lignes

--- Apres fusion avec meteo ---
Shape : (7491733, 25)
Lignes avant  : 7,491,733
Lignes apres  : 7,491,733
Lignes avec meteo : 6,724,189 (89.8%)
Lignes sans meteo : 767,544 (10.2%)

Colonnes apres fusion meteo : ['batiment_id', 'unite', 'timestamp', 'consommation', 'annee', 'mois', 'jour', 'heure', 'date', 'annee_mois', 'type_energie', 'nom', 'type', 'commune', 'surface_m

,batiment_id,unite,timestamp,consommation,annee,mois,jour,heure,date,annee_mois,type_energie,nom,type,commune,surface_m2,annee_construction,classe_energetique,nb_occupants_moyen,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm,saison
0,BAT0001,m3,2023-01-01 02:00:00,0.2600,2023,1,1,3,2023-01-01,2023-01,eau,Ecole Paris 1,ecole,Paris,1926,1978,E,225,NaN,NaN,NaN,NaN,NaN,NaN
1,BAT0001,m3,2023-01-01 17:00:00,2.2700,2023,1,1,18,2023-01-01,2023-01,eau,Ecole Paris 1,ecole,Paris,1926,1978,E,225,0.9000,78.3000,534.3000,32.0000,0.0000,hiver
2,BAT0001,m3,2023-01-01 22:00:00,0.2000,2023,1,1,23,2023-01-01,2023-01,eau,Ecole Paris 1,ecole,Paris,1926,1978,E,225,2.3000,74.2000,34.6000,18.5000,12.5000,hiver
3,BAT0001,m3,2023-01-02 21:00:00,14.6400,2023,1,2,22,2023-01-02,2023-01,eau,Ecole Paris 1,ecole,Paris,1926,1978,E,225,1.5000,60.5000,27.1000,28.7000,0.4000,hiver
4,BAT0001,m3,2023-01-03 08:00:00,14.8100,2023,1,3,9,2023-01-03,2023-01,eau,Ecole Paris 1,ecole,Paris,1926,1978,E,225,7.0000,46.7000,130.1000,9.4000,0.0000,hiver


## 4. Fusion avec les tarifs pour calculer le cout

Pour chaque ligne de consommation, on recherche le tarif applicable en fonction
du `type_energie` et de la date (dans l'intervalle `[date_debut, date_fin]`).
Le cout est ensuite calcule : `cout = consommation * tarif_unitaire`.

In [5]:
# ============================================================
# Attribution du tarif unitaire selon type_energie et date
# ============================================================
print("Tarifs disponibles :")
display(df_tarifs)

# Extraction de la date a partir du timestamp
df_merged['date'] = df_merged['timestamp'].dt.normalize()

# Fonction d'attribution du tarif pour chaque ligne
def attribuer_tarif(row, tarifs):
    """Trouve le tarif applicable selon le type d'energie et la date."""
    masque = (
        (tarifs['type_energie'] == row['type_energie']) &
        (tarifs['date_debut'] <= row['date']) &
        (tarifs['date_fin'] >= row['date'])
    )
    resultat = tarifs.loc[masque, 'tarif_unitaire']
    if len(resultat) > 0:
        return resultat.values[0]
    return np.nan

# Methode optimisee : jointure conditionnelle via cross join filtre
# Plus performant que apply() pour de gros volumes
print("\nAttribution des tarifs en cours (methode optimisee)...")

# Creer une colonne date normalisee dans le merged
df_merged['date_norm'] = pd.to_datetime(df_merged['timestamp'].dt.date)

# Pour chaque type d'energie, appliquer les tranches tarifaires
df_merged['tarif_unitaire'] = np.nan

for _, tarif_row in df_tarifs.iterrows():
    masque = (
        (df_merged['type_energie'] == tarif_row['type_energie']) &
        (df_merged['date_norm'] >= tarif_row['date_debut']) &
        (df_merged['date_norm'] <= tarif_row['date_fin'])
    )
    df_merged.loc[masque, 'tarif_unitaire'] = tarif_row['tarif_unitaire']

# Calcul du cout
df_merged['cout'] = df_merged['consommation'] * df_merged['tarif_unitaire']

# Suppression des colonnes temporaires
df_merged.drop(columns=['date', 'date_norm'], inplace=True)

# Verification
nb_sans_tarif = df_merged['tarif_unitaire'].isna().sum()
print(f"\nLignes sans tarif trouve : {nb_sans_tarif:,} ({nb_sans_tarif / len(df_merged) * 100:.2f}%)")
print(f"\nStatistiques des couts :")
print(df_merged.groupby('type_energie')[['tarif_unitaire', 'consommation', 'cout']].describe().round(4))

print(f"\nApercu avec couts :")
display(df_merged[['batiment_id', 'timestamp', 'type_energie', 'consommation', 'unite',
                    'tarif_unitaire', 'cout']].head(10))

Tarifs disponibles :


,date_debut,date_fin,type_energie,tarif_unitaire
0,2023-01-01,2023-06-30,electricite,0.1800
1,2023-07-01,2023-12-31,electricite,0.2000
2,2023-01-01,2023-06-30,gaz,0.0900
3,2023-07-01,2023-12-31,gaz,0.1000
4,2023-01-01,2023-12-31,eau,3.5000
5,2024-01-01,2024-06-30,electricite,0.2100
6,2024-07-01,2024-12-31,electricite,0.2200
7,2024-01-01,2024-06-30,gaz,0.1100
8,2024-07-01,2024-12-31,gaz,0.1200
9,2024-01-01,2024-12-31,eau,3.7500



Attribution des tarifs en cours (methode optimisee)...

Lignes sans tarif trouve : 433 (0.01%)

Statistiques des couts :
             tarif_unitaire                                                   \
                      count   mean    std    min    25%    50%    75%    max   
type_energie                                                                   
eau            2497085.0000 3.6252 0.1250 3.5000 3.5000 3.7500 3.7500 3.7500   
electricite    2497080.0000 0.2026 0.0148 0.1800 0.2000 0.2100 0.2200 0.2200   
gaz            2497135.0000 0.1050 0.0112 0.0900 0.1000 0.1100 0.1200 0.1200   

             consommation                                                     \
                    count     mean      std    min     25%      50%      75%   
type_energie                                                                   
eau          2497228.0000  42.7774  90.3152 0.0200  1.7500   7.5200  22.5600   
electricite  2497225.0000 270.9631 425.2693 0.2300 29.8200 108.6000 302.7400 

,batiment_id,timestamp,type_energie,consommation,unite,tarif_unitaire,cout
0,BAT0001,2023-01-01 02:00:00,eau,0.2600,m3,3.5000,0.9100
1,BAT0001,2023-01-01 17:00:00,eau,2.2700,m3,3.5000,7.9450
2,BAT0001,2023-01-01 22:00:00,eau,0.2000,m3,3.5000,0.7000
3,BAT0001,2023-01-02 21:00:00,eau,14.6400,m3,3.5000,51.2400
4,BAT0001,2023-01-03 08:00:00,eau,14.8100,m3,3.5000,51.8350
5,BAT0001,2023-01-03 09:00:00,eau,16.4400,m3,3.5000,57.5400
6,BAT0001,2023-01-03 11:00:00,eau,14.8300,m3,3.5000,51.9050
7,BAT0001,2023-01-03 13:00:00,eau,11.9400,m3,3.5000,41.7900
8,BAT0001,2023-01-04 22:00:00,eau,1.0400,m3,3.5000,3.6400
9,BAT0001,2023-01-05 05:00:00,eau,11.6200,m3,3.5000,40.6700


## 5. Creation des features derivees

Enrichissement du dataset avec des indicateurs calcules :
- **consommation_par_occupant** : consommation rapportee au nombre moyen d'occupants
- **consommation_par_m2** : consommation rapportee a la surface
- **IPE** (Indice de Performance Energetique) : consommation totale / surface du batiment
- **ecart_moyenne_categorie** : ecart de la consommation moyenne du batiment par rapport a sa categorie (type)

In [6]:
# ============================================================
# 5.1 Features de consommation normalisee
# ============================================================

# Consommation par occupant
df_merged['consommation_par_occupant'] = (
    df_merged['consommation'] / df_merged['nb_occupants_moyen']
)

# Consommation par m2
df_merged['consommation_par_m2'] = (
    df_merged['consommation'] / df_merged['surface_m2']
)

print("Features de consommation normalisee creees.")
print(f"  - consommation_par_occupant : min={df_merged['consommation_par_occupant'].min():.4f}, "
      f"max={df_merged['consommation_par_occupant'].max():.4f}, "
      f"moy={df_merged['consommation_par_occupant'].mean():.4f}")
print(f"  - consommation_par_m2 : min={df_merged['consommation_par_m2'].min():.6f}, "
      f"max={df_merged['consommation_par_m2'].max():.6f}, "
      f"moy={df_merged['consommation_par_m2'].mean():.6f}")

# ============================================================
# 5.2 IPE - Indice de Performance Energetique
# IPE = consommation totale du batiment / surface_m2
# ============================================================
ipe_par_batiment = (
    df_merged.groupby('batiment_id')['consommation'].sum()
    / df_merged.groupby('batiment_id')['surface_m2'].first()
)
ipe_par_batiment.name = 'ipe'

# Joindre l'IPE au dataset principal
df_merged = df_merged.merge(
    ipe_par_batiment.reset_index(),
    on='batiment_id',
    how='left'
)

print(f"\nIPE (Indice de Performance Energetique) par batiment :")
ipe_resume = df_merged.groupby(['batiment_id', 'nom', 'type', 'classe_energetique'])['ipe'].first().reset_index()
ipe_resume = ipe_resume.sort_values('ipe', ascending=False)
display(ipe_resume.head(10))

# ============================================================
# 5.3 Ecart a la moyenne de la categorie (type de batiment)
# ============================================================
# Calcul de la consommation moyenne par type de batiment
conso_moy_par_type = df_merged.groupby('type')['consommation'].mean()
conso_moy_par_type.name = 'conso_moyenne_categorie'

# Calcul de la consommation moyenne par batiment
conso_moy_par_batiment = df_merged.groupby(['batiment_id', 'type'])['consommation'].mean().reset_index()
conso_moy_par_batiment.columns = ['batiment_id', 'type', 'conso_moyenne_batiment']

# Jointure avec la moyenne categorie
conso_moy_par_batiment = conso_moy_par_batiment.merge(
    conso_moy_par_type.reset_index(),
    on='type',
    how='left'
)
conso_moy_par_batiment['ecart_moyenne_categorie'] = (
    conso_moy_par_batiment['conso_moyenne_batiment'] - conso_moy_par_batiment['conso_moyenne_categorie']
)

# Joindre au dataset principal
df_merged = df_merged.merge(
    conso_moy_par_batiment[['batiment_id', 'ecart_moyenne_categorie']],
    on='batiment_id',
    how='left'
)

print(f"\nEcart a la moyenne par categorie :")
ecart_resume = df_merged.groupby(['batiment_id', 'nom', 'type'])['ecart_moyenne_categorie'].first().reset_index()
ecart_resume = ecart_resume.sort_values('ecart_moyenne_categorie', ascending=False)
display(ecart_resume.head(10))

# ============================================================
# 5.4 Cout journalier (aggrege par batiment et jour)
# ============================================================
df_merged['date_jour'] = df_merged['timestamp'].dt.date

cout_journalier = (
    df_merged.groupby(['batiment_id', 'date_jour'])['cout']
    .sum()
    .reset_index()
    .rename(columns={'cout': 'cout_journalier'})
)

# Joindre le cout journalier
df_merged = df_merged.merge(
    cout_journalier,
    on=['batiment_id', 'date_jour'],
    how='left'
)

# Suppression de la colonne temporaire
df_merged.drop(columns=['date_jour'], inplace=True)

print(f"\nCout journalier : min={df_merged['cout_journalier'].min():.2f}, "
      f"max={df_merged['cout_journalier'].max():.2f}, "
      f"moy={df_merged['cout_journalier'].mean():.2f}")

# ============================================================
# Resume des features creees
# ============================================================
print(f"\n{'='*60}")
print(f"RESUME DES FEATURES DERIVEES")
print(f"{'='*60}")
features_derivees = ['consommation_par_occupant', 'consommation_par_m2', 'ipe',
                      'ecart_moyenne_categorie', 'cout_journalier']
for feat in features_derivees:
    if feat in df_merged.columns:
        print(f"  {feat:35s} : moy={df_merged[feat].mean():.4f}, "
              f"NaN={df_merged[feat].isna().sum():,} ({df_merged[feat].isna().mean()*100:.1f}%)")

print(f"\nApercu du dataset enrichi :")
display(df_merged[['batiment_id', 'nom', 'type_energie', 'consommation',
                    'consommation_par_occupant', 'consommation_par_m2',
                    'ipe', 'ecart_moyenne_categorie', 'cout', 'cout_journalier']].head(10))

Features de consommation normalisee creees.
  - consommation_par_occupant : min=0.0002, max=42.5766, moy=1.4743
  - consommation_par_m2 : min=0.000024, max=1.622360, moy=0.112455

IPE (Indice de Performance Energetique) par batiment :


,batiment_id,nom,type,classe_energetique,ipe
121,BAT0122,Piscine Le Havre 122,piscine,G,17032.6556
120,BAT0121,Piscine Le Havre 121,piscine,G,16990.0054
4,BAT0005,Piscine Paris 5,piscine,G,16981.1561
42,BAT0043,Piscine Bordeaux 43,piscine,G,16976.4134
133,BAT0134,Piscine Saint-Etienne 134,piscine,G,16966.3154
111,BAT0112,Piscine Reims 112,piscine,G,16960.5976
47,BAT0048,Piscine Lille 48,piscine,G,16930.1142
145,BAT0146,Piscine Toulon 146,piscine,F,14499.6984
49,BAT0050,Piscine Lille 50,piscine,F,14460.0947
91,BAT0092,Piscine Nice 92,piscine,F,14458.9924



Ecart a la moyenne par categorie :


,batiment_id,nom,type,ecart_moyenne_categorie
4,BAT0005,Piscine Paris 5,piscine,598.8403
47,BAT0048,Piscine Lille 48,piscine,544.8276
135,BAT0136,Piscine Toulon 136,piscine,408.4448
111,BAT0112,Piscine Reims 112,piscine,328.3033
132,BAT0133,Piscine Saint-Etienne 133,piscine,290.1972
49,BAT0050,Piscine Lille 50,piscine,280.6643
142,BAT0143,Piscine Toulon 143,piscine,241.6182
18,BAT0019,Piscine Lyon 19,piscine,181.2393
120,BAT0121,Piscine Le Havre 121,piscine,178.0297
13,BAT0014,Gymnase Lyon 14,gymnase,163.7156



Cout journalier : min=0.00, max=49912.73, moy=5909.24

RESUME DES FEATURES DERIVEES
  consommation_par_occupant           : moy=1.4743, NaN=0 (0.0%)
  consommation_par_m2                 : moy=0.1125, NaN=0 (0.0%)
  ipe                                 : moy=5770.4702, NaN=0 (0.0%)
  ecart_moyenne_categorie             : moy=-0.0000, NaN=0 (0.0%)
  cout_journalier                     : moy=5909.2412, NaN=0 (0.0%)

Apercu du dataset enrichi :


,batiment_id,nom,type_energie,consommation,consommation_par_occupant,consommation_par_m2,ipe,ecart_moyenne_categorie,cout,cout_journalier
0,BAT0001,Ecole Paris 1,eau,0.2600,0.0012,0.0001,3189.1249,3.3344,0.9100,568.2182
1,BAT0001,Ecole Paris 1,eau,2.2700,0.0101,0.0012,3189.1249,3.3344,7.9450,568.2182
2,BAT0001,Ecole Paris 1,eau,0.2000,0.0009,0.0001,3189.1249,3.3344,0.7000,568.2182
3,BAT0001,Ecole Paris 1,eau,14.6400,0.0651,0.0076,3189.1249,3.3344,51.2400,2535.4485
4,BAT0001,Ecole Paris 1,eau,14.8100,0.0658,0.0077,3189.1249,3.3344,51.8350,2783.5466
5,BAT0001,Ecole Paris 1,eau,16.4400,0.0731,0.0085,3189.1249,3.3344,57.5400,2783.5466
6,BAT0001,Ecole Paris 1,eau,14.8300,0.0659,0.0077,3189.1249,3.3344,51.9050,2783.5466
7,BAT0001,Ecole Paris 1,eau,11.9400,0.0531,0.0062,3189.1249,3.3344,41.7900,2783.5466
8,BAT0001,Ecole Paris 1,eau,1.0400,0.0046,0.0005,3189.1249,3.3344,3.6400,2714.5248
9,BAT0001,Ecole Paris 1,eau,11.6200,0.0516,0.0060,3189.1249,3.3344,40.6700,2851.6479


## 6. Verification de coherence

Controles qualite sur le dataset enrichi final :
- Pourcentage de valeurs manquantes par colonne
- Validation des plages de valeurs
- Statistiques descriptives
- Qualite des jointures

In [7]:
# ============================================================
# 6.1 Pourcentage de valeurs manquantes par colonne
# ============================================================
print("=" * 60)
print("VERIFICATION DE COHERENCE DU DATASET ENRICHI")
print("=" * 60)

print(f"\nShape final : {df_merged.shape}")
print(f"Memoire totale : {df_merged.memory_usage(deep=True).sum() / 1e6:.1f} Mo")

print(f"\n--- Valeurs manquantes (NaN) ---")
nan_stats = pd.DataFrame({
    'nb_nan': df_merged.isna().sum(),
    'pct_nan': (df_merged.isna().mean() * 100).round(2)
}).sort_values('pct_nan', ascending=False)
nan_stats = nan_stats[nan_stats['nb_nan'] > 0]
if len(nan_stats) > 0:
    display(nan_stats)
else:
    print("Aucune valeur manquante !")

# ============================================================
# 6.2 Validation des plages de valeurs
# ============================================================
print(f"\n--- Validation des plages de valeurs ---")

controles = []

# Consommation positive
nb_conso_neg = (df_merged['consommation'] < 0).sum()
controles.append(('Consommation >= 0', nb_conso_neg == 0, f"{nb_conso_neg:,} valeurs negatives"))

# Cout positif (ou NaN)
nb_cout_neg = (df_merged['cout'] < 0).sum()
controles.append(('Cout >= 0', nb_cout_neg == 0, f"{nb_cout_neg:,} valeurs negatives"))

# Surface positive
nb_surface_neg = (df_merged['surface_m2'] <= 0).sum()
controles.append(('Surface > 0', nb_surface_neg == 0, f"{nb_surface_neg:,} valeurs <= 0"))

# Temperature dans une plage raisonnable
if 'temperature_c' in df_merged.columns:
    temp_hors_plage = ((df_merged['temperature_c'] < -30) | (df_merged['temperature_c'] > 50)).sum()
    controles.append(('Temperature [-30, 50]', temp_hors_plage == 0, f"{temp_hors_plage:,} hors plage"))

# Nombre d'occupants positif
nb_occ_neg = (df_merged['nb_occupants_moyen'] <= 0).sum()
controles.append(('Nb occupants > 0', nb_occ_neg == 0, f"{nb_occ_neg:,} valeurs <= 0"))

# Tarif unitaire positif
nb_tarif_neg = (df_merged['tarif_unitaire'].dropna() < 0).sum()
controles.append(('Tarif >= 0', nb_tarif_neg == 0, f"{nb_tarif_neg:,} valeurs negatives"))

for nom_controle, ok, detail in controles:
    statut = "OK" if ok else "ATTENTION"
    print(f"  [{statut:9s}] {nom_controle:30s} {detail}")

# ============================================================
# 6.3 Statistiques descriptives
# ============================================================
print(f"\n--- Statistiques descriptives (colonnes numeriques) ---")
cols_num = df_merged.select_dtypes(include=[np.number]).columns
display(df_merged[cols_num].describe().round(4))

# ============================================================
# 6.4 Qualite des jointures
# ============================================================
print(f"\n--- Qualite des jointures ---")
print(f"Jointure batiments :")
print(f"  - Lignes avec commune renseignee : {df_merged['commune'].notna().sum():,} / {len(df_merged):,}")
print(f"  - Taux : {df_merged['commune'].notna().mean() * 100:.2f}%")

print(f"\nJointure meteo :")
if 'temperature_c' in df_merged.columns:
    print(f"  - Lignes avec temperature renseignee : {df_merged['temperature_c'].notna().sum():,} / {len(df_merged):,}")
    print(f"  - Taux : {df_merged['temperature_c'].notna().mean() * 100:.2f}%")

print(f"\nJointure tarifs :")
print(f"  - Lignes avec tarif renseigne : {df_merged['tarif_unitaire'].notna().sum():,} / {len(df_merged):,}")
print(f"  - Taux : {df_merged['tarif_unitaire'].notna().mean() * 100:.2f}%")

print(f"\n--- Repartition par type d'energie ---")
print(df_merged['type_energie'].value_counts())

print(f"\n--- Repartition par type de batiment ---")
print(df_merged['type'].value_counts())

VERIFICATION DE COHERENCE DU DATASET ENRICHI

Shape final : (7491733, 30)
Memoire totale : 4489.7 Mo

--- Valeurs manquantes (NaN) ---


,nb_nan,pct_nan
saison,767544,10.2500
precipitation_mm,767544,10.2500
vitesse_vent_kmh,767544,10.2500
rayonnement_solaire_wm2,767544,10.2500
humidite_pct,767544,10.2500
temperature_c,767544,10.2500
cout,433,0.0100
tarif_unitaire,433,0.0100



--- Validation des plages de valeurs ---
  [OK       ] Consommation >= 0              0 valeurs negatives
  [OK       ] Cout >= 0                      0 valeurs negatives
  [OK       ] Surface > 0                    0 valeurs <= 0
  [OK       ] Temperature [-30, 50]          0 hors plage
  [OK       ] Nb occupants > 0               0 valeurs <= 0
  [OK       ] Tarif >= 0                     0 valeurs negatives

--- Statistiques descriptives (colonnes numeriques) ---


,consommation,annee,mois,jour,heure,surface_m2,annee_construction,nb_occupants_moyen,temperature_c,humidite_pct,rayonnement_solaire_wm2,vitesse_vent_kmh,precipitation_mm,tarif_unitaire,cout,consommation_par_occupant,consommation_par_m2,ipe,ecart_moyenne_categorie,cout_journalier
count,7491733.0000,7491733.0000,7491733.0000,7491733.0000,7491733.0000,7491733.0000,7491733.0000,7491733.0000,6724189.0000,6724189.0000,6724189.0000,6724189.0000,6724189.0000,7491300.0000,7491300.0000,7491733.0000,7491733.0000,7491733.0000,7491733.0000,7491733.0000
mean,239.5332,2023.5007,6.5204,15.7376,11.5021,1704.0528,1986.4687,174.7732,15.1015,62.8485,257.7581,19.9761,1.8822,1.3109,84.1409,1.4743,0.1125,5770.4702,-0.0000,5909.2412
std,470.1259,0.5000,3.4494,8.8031,6.9218,865.4189,20.0591,117.3846,8.5143,18.9445,257.3286,11.5555,3.9092,1.6385,205.7449,2.6657,0.1666,4061.0350,135.5719,8878.1754
min,0.0200,2023.0000,1.0000,1.0000,0.0000,438.0000,1950.0000,20.0000,-4.0000,30.0000,0.0000,0.0000,0.0000,0.0900,0.0315,0.0002,0.0000,919.7430,-446.9393,0.0000
25%,10.2000,2023.0000,4.0000,8.0000,6.0000,1126.0000,1972.0000,78.0000,8.9000,46.5000,29.7000,10.0000,0.0000,0.1200,5.5692,0.0838,0.0078,3128.2913,-42.4993,826.6382
50%,56.5600,2024.0000,7.0000,16.0000,12.0000,1561.0000,1985.0000,140.0000,14.6000,62.9000,157.3000,19.9000,0.0000,0.2100,21.1222,0.4161,0.0384,4187.2560,-8.2312,1910.2840
75%,245.0700,2024.0000,10.0000,23.0000,18.0000,2146.0000,2003.0000,256.0000,21.1000,79.1000,476.5000,30.0000,0.0000,3.5000,62.0950,1.8030,0.1556,6838.2199,42.8705,5234.3990
max,6348.1900,2024.0000,12.0000,31.0000,23.0000,3942.0000,2020.0000,495.0000,36.0000,100.0000,800.0000,40.0000,15.0000,3.7500,2479.8000,42.5766,1.6224,17032.6556,598.8403,49912.7346



--- Qualite des jointures ---
Jointure batiments :
  - Lignes avec commune renseignee : 7,491,733 / 7,491,733
  - Taux : 100.00%

Jointure meteo :
  - Lignes avec temperature renseignee : 6,724,189 / 7,491,733
  - Taux : 89.75%

Jointure tarifs :
  - Lignes avec tarif renseigne : 7,491,300 / 7,491,733
  - Taux : 99.99%

--- Repartition par type d'energie ---
type_energie
gaz            2497280
eau            2497228
electricite    2497225
Name: count, dtype: int64

--- Repartition par type de batiment ---
type
piscine        1590925
mairie         1590542
mediatheque    1539374
gymnase        1385504
ecole          1385388
Name: count, dtype: int64


## 7. Dictionnaire de donnees

Description exhaustive de toutes les colonnes du dataset enrichi final.

In [8]:
# ============================================================
# Dictionnaire de donnees du dataset enrichi
# ============================================================

dictionnaire = {
    'batiment_id':              ('object',   'Identifiant unique du batiment',                 '-'),
    'timestamp':                ('datetime', 'Horodatage de la mesure',                       'YYYY-MM-DD HH:MM:SS'),
    'type_energie':             ('object',   'Type d\'energie (electricite, gaz, eau)',        '-'),
    'consommation':             ('float64',  'Valeur de consommation mesuree',                'kWh / m3'),
    'unite':                    ('object',   'Unite de la consommation',                      '-'),
    'nom':                      ('object',   'Nom du batiment',                               '-'),
    'type':                     ('object',   'Categorie du batiment (ecole, mairie, etc.)',   '-'),
    'commune':                  ('object',   'Commune de localisation du batiment',           '-'),
    'surface_m2':               ('int64',    'Surface totale du batiment',                    'm2'),
    'annee_construction':       ('int64',    'Annee de construction du batiment',             'annee'),
    'classe_energetique':       ('object',   'Classe energetique DPE (A a G)',                '-'),
    'nb_occupants_moyen':       ('int64',    'Nombre moyen d\'occupants',                    'personnes'),
    'temperature_c':            ('float64',  'Temperature exterieure',                        'degres C'),
    'humidite_pct':             ('float64',  'Humidite relative',                             '%'),
    'rayonnement_solaire_wm2':  ('float64',  'Rayonnement solaire',                           'W/m2'),
    'vitesse_vent_kmh':         ('float64',  'Vitesse du vent',                               'km/h'),
    'precipitation_mm':         ('float64',  'Precipitations',                                'mm'),
    'saison':                   ('object',   'Saison (printemps, ete, automne, hiver)',       '-'),
    'tarif_unitaire':           ('float64',  'Tarif unitaire applicable',                     'EUR/kWh ou EUR/m3'),
    'cout':                     ('float64',  'Cout = consommation x tarif_unitaire',          'EUR'),
    'consommation_par_occupant':('float64',  'Consommation / nb_occupants_moyen',             'kWh/pers ou m3/pers'),
    'consommation_par_m2':      ('float64',  'Consommation / surface_m2',                     'kWh/m2 ou m3/m2'),
    'ipe':                      ('float64',  'Indice de Performance Energetique (total/m2)',  'kWh/m2'),
    'ecart_moyenne_categorie':  ('float64',  'Ecart conso moyenne batiment vs categorie',     'kWh ou m3'),
    'cout_journalier':          ('float64',  'Cout total journalier par batiment',            'EUR/jour'),
}

# Construire le DataFrame du dictionnaire
dict_df = pd.DataFrame(
    [(col, info[0], info[1], info[2]) for col, info in dictionnaire.items()],
    columns=['Colonne', 'Type', 'Description', 'Unite']
)

# Marquer les colonnes presentes / absentes dans le dataset
dict_df['Present'] = dict_df['Colonne'].apply(lambda c: 'Oui' if c in df_merged.columns else 'Non')

print("=" * 80)
print("DICTIONNAIRE DE DONNEES - Dataset enrichi")
print(f"Nombre total de colonnes dans le dataset : {len(df_merged.columns)}")
print("=" * 80)

# Affichage formate
pd.set_option('display.max_colwidth', 60)
display(dict_df)

# Colonnes du dataset non documentees
cols_non_doc = set(df_merged.columns) - set(dictionnaire.keys())
if cols_non_doc:
    print(f"\nColonnes presentes dans le dataset mais non documentees : {cols_non_doc}")
else:
    print(f"\nToutes les colonnes du dataset sont documentees.")

DICTIONNAIRE DE DONNEES - Dataset enrichi
Nombre total de colonnes dans le dataset : 30


,Colonne,Type,Description,Unite,Present
0,batiment_id,object,Identifiant unique du batiment,-,Oui
1,timestamp,datetime,Horodatage de la mesure,YYYY-MM-DD HH:MM:SS,Oui
2,type_energie,object,"Type d'energie (electricite, gaz, eau)",-,Oui
3,consommation,float64,Valeur de consommation mesuree,kWh / m3,Oui
4,unite,object,Unite de la consommation,-,Oui
5,nom,object,Nom du batiment,-,Oui
6,type,object,"Categorie du batiment (ecole, mairie, etc.)",-,Oui
7,commune,object,Commune de localisation du batiment,-,Oui
8,surface_m2,int64,Surface totale du batiment,m2,Oui
9,annee_construction,int64,Annee de construction du batiment,annee,Oui



Colonnes presentes dans le dataset mais non documentees : {'heure', 'annee_mois', 'mois', 'annee', 'jour'}


## 8. Sauvegarde

Export du dataset enrichi en deux formats :
- **CSV** : pour la compatibilite et l'inspection manuelle
- **Parquet** : pour les performances de lecture et la compression

In [ ]:
# ============================================================
# Sauvegarde du dataset enrichi
# ============================================================

# Chemins de sortie
path_csv = os.path.join(PATH_OUTPUT, 'consommations_enrichies.csv')
path_parquet = os.path.join(PATH_OUTPUT, 'consommations_enrichies.parquet')

# Creation du repertoire de sortie si necessaire
os.makedirs(PATH_OUTPUT, exist_ok=True)

# Sauvegarde CSV
print("Sauvegarde en CSV...")
df_merged.to_csv(path_csv, index=False, encoding='utf-8')
taille_csv = os.path.getsize(path_csv) / (1024 * 1024)
print(f"  -> {path_csv}")
print(f"     Taille : {taille_csv:.1f} Mo")

# Sauvegarde Parquet
print("\nSauvegarde en Parquet...")
df_merged.to_parquet(path_parquet, index=False, engine='pyarrow', compression='snappy')
taille_parquet = os.path.getsize(path_parquet) / (1024 * 1024)
print(f"  -> {path_parquet}")
print(f"     Taille : {taille_parquet:.1f} Mo")

# Comparaison des tailles
ratio = taille_csv / taille_parquet if taille_parquet > 0 else 0
print(f"\nRatio de compression CSV/Parquet : {ratio:.1f}x")

# Verification de la relecture
print("\nVerification de relecture Parquet...")
df_verif = pd.read_parquet(path_parquet)
assert df_verif.shape == df_merged.shape, "ERREUR : les shapes ne correspondent pas !"
print(f"  Shape relue : {df_verif.shape} -> OK")

print(f"\n{'='*60}")
print(f"SAUVEGARDE TERMINEE")
print(f"  - CSV     : {path_csv} ({taille_csv:.1f} Mo)")
print(f"  - Parquet : {path_parquet} ({taille_parquet:.1f} Mo)")
print(f"  - Lignes  : {df_merged.shape[0]:,}")
print(f"  - Colonnes: {df_merged.shape[1]}")
print(f"{'='*60}")

Sauvegarde en CSV...


## Conclusion

Ce notebook a realise les etapes suivantes :

1. **Chargement** des 4 sources de donnees nettoyees (consommations, batiments, meteo, tarifs)
2. **Fusion consommations-batiments** : jointure sur `batiment_id` pour enrichir chaque mesure avec les caracteristiques du batiment
3. **Fusion avec la meteo** : jointure sur `commune` + `timestamp` (arrondi a l'heure) pour ajouter les conditions climatiques
4. **Calcul des couts** : application des tarifs par type d'energie et periode, puis calcul du cout unitaire
5. **Features derivees** :
   - `consommation_par_occupant` et `consommation_par_m2` : normalisation de la consommation
   - `ipe` (Indice de Performance Energetique) : consommation totale rapportee a la surface
   - `ecart_moyenne_categorie` : positionnement du batiment par rapport a sa categorie
   - `cout_journalier` : agregation quotidienne des couts
6. **Verification de coherence** : controles qualite, taux de NaN, validation des plages
7. **Dictionnaire de donnees** : documentation exhaustive de chaque colonne
8. **Sauvegarde** en CSV et Parquet pour exploitation ulterieure

Le dataset enrichi est pret pour les etapes suivantes :
- Analyse exploratoire approfondie (EDA)
- Visualisations
- Modelisation predictive